In [94]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [95]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from cleantext import clean

In [97]:
df = pd.read_csv('/content/twitter_tweet (2).csv')
df.head()

,Unnamed: 0,Date,User,Tweet
0,0,2022-12-11 23:59:16+00:00,G285669957,"@PlayRuneterra My mony..\nCan you add ""are you..."
1,1,2022-12-11 23:59:04+00:00,chadsmith71,@hyok_lee Cut through that lemon like it was b...
2,2,2022-12-11 23:58:37+00:00,RhondaAhimsa,@thehealthb0t Uhm you don't want to buy eggs o...
3,3,2022-12-11 23:58:29+00:00,gentarotrash,@uruao1 Right?! I like the one on the second e...
4,4,2022-12-11 23:58:16+00:00,alicehatter321,@elonmusk @stillgray I’m going to buy more Tes...


#Preprocessing

In [98]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [99]:
def lemmatizing(words):
    lemmatizer =WordNetLemmatizer()
    return ''.join([lemmatizer.lemmatize(word) for word in words])
df['Tweet']=df['Tweet'].apply(lambda x: lemmatizing(str(x)))

In [100]:
with open('slang.txt') as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True) # longest first for regex
slang_regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))

In [101]:
df['Tweet']

0        @PlayRuneterra My mony..\nCan you add "are you...
1        @hyok_lee Cut through that lemon like it was b...
2        @thehealthb0t Uhm you don't want to buy eggs o...
3        @uruao1 Right?! I like the one on the second e...
4        @elonmusk @stillgray I’m going to buy more Tes...
                               ...                        
49995    I know They're crying in the comment section. ...
49996    But Ulez is also going to affect low-paid NHS ...
49997    You can't go wrong with perfume as a gift.\nIf...
49998    @deejaybugy @SafeBoda Your candles actually wo...
49999    I need to buy a new couch that's incredibly un...
Name: Tweet, Length: 50000, dtype: object

In [102]:
def tweet_preprocess(tweet):

  tweet = re.sub(r'^RT[\s]+', '', tweet)
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
  tweet = re.sub(r'#', '', tweet)
  tweet = re.sub(r'[0-9]', '', tweet)
  tweet = re.sub(r'@[^ ]+', '', tweet)
  tweet = re.sub(r'(buy|sell|need|want)(to)*', '', tweet)
  tweet = re.sub(r'(time|day|week|month|year)(s)*', '', tweet)
  tweet = re.sub(r'(man|lot|people|shit|venus|ina|dm|family|sir|wtb|mom|help|want|wts|season|merch|boss|thank|cause|hey|guess|andamp|one|work|thing|way)(s)*', '', tweet)

  tweet = re.sub(slang_regex,'',tweet)
  tweet = clean(tweet, no_emoji=True)
   
  nouns = []
  for word,pos in nltk.pos_tag(nltk.word_tokenize(str(tweet))):
    if (pos == 'NN') and len(word) > 2:
      nouns.append(word)
  if len(nouns) > 0:
    return ' '.join(nouns)
  else:
    return ''

In [103]:
df['Tweet_processed'] = df['Tweet'].apply(tweet_preprocess)
df['Tweet_processed']

0                                                     mony
1            butter pressure hand chef knife brand quality
2                uhm egg industry baby male blender ground
3                                               extra haha
4                               stock mon share kind tweet
                               ...                        
49995             comment section palm produce stock .....
49996       ulez staff transport car pressure care england
49997    perfume gift perfume xmasgiftidea christmasgif...
49998                                                     
49999                                                couch
Name: Tweet_processed, Length: 50000, dtype: object

In [104]:
df.head(20)

,Date,User,Tweet,Tweet_processed
0,2022-12-11 23:59:16+00:00,G285669957,"@PlayRuneterra My mony..\nCan you add ""are you...",mony
1,2022-12-11 23:59:04+00:00,chadsmith71,@hyok_lee Cut through that lemon like it was b...,butter pressure hand chef knife brand quality
2,2022-12-11 23:58:37+00:00,RhondaAhimsa,@thehealthb0t Uhm you don't want to buy eggs o...,uhm egg industry baby male blender ground
3,2022-12-11 23:58:29+00:00,gentarotrash,@uruao1 Right?! I like the one on the second e...,extra haha
4,2022-12-11 23:58:16+00:00,alicehatter321,@elonmusk @stillgray I’m going to buy more Tes...,stock mon share kind tweet
5,2022-12-11 23:56:03+00:00,SixkillerHenson,@catturd2 If you dont want to buy women drinks...,yea cocktail
6,2022-12-11 23:56:01+00:00,Yapiyipiyehey,Badly want to buy shoes on my birthday.💔,birth
7,2022-12-11 23:56:00+00:00,crunchiebaby,THIS MAKES ME WANT TO BUY THEMMM 😭😭😭😭😭 https:/...,
8,2022-12-11 23:55:17+00:00,JillaneKent,@Knocksious I'm buying him new girly home acce...,home consumer gift maji household
9,2022-12-11 23:55:00+00:00,fejimanz,I think I want to buy myself a game boy. Get W...,game boy wario tmnt


In [105]:
drop_row = df[df['Tweet_processed'] == ''].index
df.drop(drop_row, axis =0, inplace = True)
df.shape

(44318, 4)

In [106]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=100, stop_words="english")
dtm = cv.fit_transform(df['Tweet_processed'])

dtm

<44318x266 sparse matrix of type '<class 'numpy.int64'>'
	with 60992 stored elements in Compressed Sparse Row format>

In [107]:
feature_names = cv.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [108]:
from sklearn.decomposition import LatentDirichletAllocation

NB_TOPICS = 5

LDA_model = LatentDirichletAllocation(n_components = NB_TOPICS, max_iter = 30, random_state = 0)

LDA_model.fit(dtm)

LatentDirichletAllocation(max_iter=30, n_components=5, random_state=0)

In [109]:
for i, topic in enumerate(LDA_model.components_):
  print("THE TOP {} WORDS FOR TOPIC #{}".format(10, i))
  print([cv.get_feature_names()[index] for index in topic.argsort()[-3:]])
  print("\n")

THE TOP 10 WORDS FOR TOPIC #0
['machine', 'bank', 'ticket']


THE TOP 10 WORDS FOR TOPIC #1
['stuff', 'christmas', 'food']


THE TOP 10 WORDS FOR TOPIC #2
['wtb', 'house', 'game']


THE TOP 10 WORDS FOR TOPIC #3
['album', 'school', 'car']


THE TOP 10 WORDS FOR TOPIC #4
['corporation', 'book', 'want']




In [110]:
final_topics = LDA_model.transform(dtm)

print(final_topics.shape)

df["Topic"] = final_topics.argmax(axis=1)


(44318, 5)


In [111]:
df.to_csv('/content/tweet_topic.csv')